Un papá debe decidir todos los días hábiles de la semana (lunes a viernes) si lleva, o no, a su hija al jardín. 

Si la niña va sana al jardín, la probabilidad de que amanezca enferma al día siguiente es 0.3 y si la niña va enferma al jardín, la probabilidad que siga enferma al día siguiente es 0.8. Cada día que amanece enferma, la familia debe gastar \$5.000 en medicamentos (incluso los fines de semana). 

En caso de que el papá elija no llevar a su hija al jardín (independiente de si está enferma o sana), debe pagar a una persona cuidadora un costo de \$10.000 por el día. Cuando la niña se queda en casa, con seguridad amanece sana al día siguiente (independiente de si estaba sana o enferma).

Se desea estudiar la situación para los próximos K días de la semana, siendo el K-ésimo el día viernes, sabiendo que el sábado la familia debe irse de viaje. En caso que la hija amanezca enferma el día sábado, la familia no podrá viajar lo que representa un costo de \$20.000 (que no incluye los medicamentos).

Para esto se ha formulado el siguiente modelo de programación dínamica

**Etapas**

$K$, una por cada día.

**Variables de decisión**:

$$
X_n= \left\{\begin{array}{cl}
	1& \text{si la niña asiste al jardín el día $n$} \\
	0& \text{si la niña no asiste al jardín el día $n$} \\
\end{array}\right.
$$

**Variables de estado**: 

$$S_n= \left\{\begin{array}{cl}
	1& \text{si la niña amanece sana el día $n$} \\
	0& \text{si la niña amanece enferma el día $n$} \\
\end{array}\right.$$

**Reglas de transformación** 

$S_{n+1}=\left\{\begin{array}{ccl}
	1& \text{si} & X_n=0 \\
	1& \text{si} & X_n=1, S_n=1, \text{ con } p=0.3 \\
	0& \text{si} & X_n=1, S_n=1, \text{ con } p=0.7 \\
	1& \text{si} & X_n=1, S_n=0, \text{ con } p=0.2 \\
	0& \text{si} & X_n=1, S_n=0, \text{ con } p=0.8 \\    
\end{array}\right.$ 

**Función de recompensa**

$R_n(S_n,X_n) = 5000(1-S_n)+10000(1-X_n)$

**Función recursiva**

$f_n(S_n,X_n)=\mathbb{E}(R_n(S_n,X_n)+f_{n+1}^*(S_{n+1}))$, para todo $n$.

**Función objetivo** 

$\displaystyle f_n^*(S_n)=\min_{X_n} (f_n(S_n,X_n))$ para todo $n$.

**Condiciones**

$f_{K+1}^*(S_{K+1})=25000(1-S_{K+1})$

**Restricciones**

$X_n, S_n \geq 0$, enteras, para todo $n$.



En primer lugar necesitamos definir tres funciones: la función de transformación (función de actualización de la variable de estado), función de recompensa y la función recursiva.

Para las etapas usaremos un arreglo de tres dimensiones, la primera dimensión corresponderá a las distinas etapas, la segunda a los valores de la variable de estado y la tercera a los valores de la variable de decisión. Como en este caso es posible asignar cualquier cantidad de equipos médicos a un país (respetando las restricciones) las filas van de $0$ a $R$, al igual que las columnas.

Asignaremos el valor -1 para inicializar las matrices de las etapas, a fin de poder determinar aquellas celdas infactibles.

In [1]:
K=5#lunes,martes,mier,jueve,viern
etapas=K+1#lunes,martes,mier,jueve,viern,sabado

decisiones=2 #0 no asiste, 1 si asiste

estados=2 #0 si esta enferma, 1 si esta sana


In [2]:
range(etapas)

range(0, 6)

In [3]:
def funcion_recompensa(S,X):
    recompensa=5000*(1-S)+10000*(1-X)
    return recompensa

In [4]:
import numpy as np

In [5]:
for i in reversed(range(etapas)):
    print(i)
print('\nEtapas:',etapas)

5
4
3
2
1
0

Etapas: 6


In [12]:
K=3#lunes,martes,mier,jueve,viern
etapas=K+1#lunes,martes,mier,jueve,viern,sabado

In [13]:
ETAPAS=[]

for i in reversed(range(etapas)):
    tabla=np.ones((estados,decisiones+2))*float('inf')
    for s in range(estados):
        for x in range(decisiones):
            if i==etapas-1: #sabado
                tabla[s,x]=25000*(1-s)
            else: #lunes, martes, miercoles, jueves, viernes
                if x==0:# si no asiste
                    optimoAnteriorEsp=1*ETAPAS[0][1,decisiones+1]
                    tabla[s,x]=funcion_recompensa(s,x)+optimoAnteriorEsp
                if x==1 and s==1:
                    optimoAnteriorEsp=0.3*ETAPAS[0][1,decisiones+1]+0.7*ETAPAS[0][0,decisiones+1]
                    tabla[s,x]=funcion_recompensa(s,x)+optimoAnteriorEsp
                if x==1 and s==0:
                    optimoAnteriorEsp=0.2*ETAPAS[0][1,decisiones+1]+0.8*ETAPAS[0][0,decisiones+1]
                    tabla[s,x]=funcion_recompensa(s,x)+optimoAnteriorEsp
                    
            if tabla[s,x]<tabla[s,decisiones+1]:
                tabla[s,decisiones+1]=tabla[s,x] #reemplazo Fx por el menor valor
                tabla[s,decisiones]=x
                
                
    ETAPAS.insert(0,tabla)
            

In [14]:
from tabulate import tabulate

In [15]:
for dia,etapa in zip(['lunes','martes','miercoles','jueves','viernes','Sabado'],ETAPAS):
    print(f'\nDia {dia}\n',tabulate(etapa,headers='0 1 X* Fx*'.split()))
    


Dia lunes
     0      1    X*    Fx*
-----  -----  ----  -----
28500  22900     1  22900
23500  17350     1  17350

Dia martes
     0      1    X*    Fx*
-----  -----  ----  -----
25000  19000     1  19000
20000  13500     1  13500

Dia miercoles
     0      1    X*    Fx*
-----  -----  ----  -----
15000  25000     0  15000
10000  17500     0  10000

Dia jueves
     0      1    X*    Fx*
-----  -----  ----  -----
25000  25000     0  25000
    0      0     0      0


In [16]:
ETAPAS=[]

for i in reversed(range(etapas)):
    tabla=np.ones((estados,decisiones+2))*float('inf')
    for s in range(estados):
        for x in range(decisiones):
            if i==etapas-1: #sabado
                tabla[s,x]=25000*(1-s)
            else: #lunes, martes, miercoles, jueves, viernes
                if x==0:# si no asiste
                    optimoAnteriorEsp=1*ETAPAS[0][1,decisiones+1]
                    tabla[s,x]=funcion_recompensa(s,x)+optimoAnteriorEsp
                if x==1 and s==1:
                    optimoAnteriorEsp=0.3*ETAPAS[0][1,decisiones+1]+0.7*ETAPAS[0][0,decisiones+1]
                    tabla[s,x]=funcion_recompensa(s,x)+optimoAnteriorEsp
                if x==1 and s==0:
                    optimoAnteriorEsp=0.2*ETAPAS[0][1,decisiones+1]+0.8*ETAPAS[0][0,decisiones+1]
                    tabla[s,x]=funcion_recompensa(s,x)+optimoAnteriorEsp
                    
            if tabla[s,x]<tabla[s,decisiones+1]:
                tabla[s,decisiones+1]=tabla[s,x] #reemplazo Fx por el menor valor
                tabla[s,decisiones]=x
                
                
    ETAPAS.insert(0,tabla)

In [17]:
for dia,etapa in zip(['miercoles','jueves','viernes','Sabado'],ETAPAS):
    print(f'\nDia {dia}\n',tabulate(etapa,headers='0 1 X* Fx*'.split()))
    


Dia miercoles
     0      1    X*    Fx*
-----  -----  ----  -----
28500  22900     1  22900
23500  17350     1  17350

Dia jueves
     0      1    X*    Fx*
-----  -----  ----  -----
25000  19000     1  19000
20000  13500     1  13500

Dia viernes
     0      1    X*    Fx*
-----  -----  ----  -----
15000  25000     0  15000
10000  17500     0  10000

Dia Sabado
     0      1    X*    Fx*
-----  -----  ----  -----
25000  25000     0  25000
    0      0     0      0


In [34]:
import numpy as np
def jardin ( T ) :
  etapas = -1* np . ones (( T ,2 ,2) )
  mejores = np . zeros ((2 , T +1) )
  mejores [0 , T ] = 25000
  mejores [1 , T ] = 0
  for k in range (T -1 , -1 , -1) :
    for i in range (2) :
        for j in range (2) :
            if j ==0:
                etapas [k ,i , j ]= funcion_recompensa (i , j ) + mejores[1 , k +1]
            elif j ==1 and i ==0:
                etapas [k ,i , j ]= funcion_recompensa (i , j ) +0.8* mejores [0 , k +1]+0.2* mejores[1 , k +1]
            elif j ==1 and i ==1:
                etapas [k ,i , j ]= funcion_recompensa (i , j ) +0.3* mejores [0 , k +1]+0.7* mejores[1 , k +1]
    mejores [i , k ]= np.min( etapas [k ,i ,:])
  return etapas , mejores

In [35]:
jardin(3)

(array([[[20250.,  6050.],
         [15250.,  3675.]],
 
        [[22500.,  6500.],
         [17500.,  5250.]],
 
        [[15000., 25000.],
         [10000.,  7500.]]]),
 array([[    0.,     0.,     0., 25000.],
        [ 3675.,  5250.,  7500.,     0.]]))